In [89]:
import librosa
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'keras.wrappers'

In [23]:
# Function to extract features from an audio file
def extract_features(file_path):
    y, sr = librosa.load(file_path)
    
    # Extract features
    tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
    rms = np.mean(librosa.feature.rms(y=y))
    chromagram = np.mean(librosa.feature.chroma_stft(y=y, sr=sr).T, axis=0)
    mel_spectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T, axis=0)
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr).T, axis=0)
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    zero_crossing_rate = np.mean(librosa.feature.zero_crossing_rate(y))
    harmonics = np.mean(librosa.effects.harmonic(y))
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr).T, axis=0)
    
    return [tempo, rms, *chromagram, *mel_spectrogram, spectral_centroid, *spectral_contrast, spectral_rolloff, zero_crossing_rate, harmonics, *mfccs]

In [24]:
dataset_directory = '/home/jovyan/teaching_material/MScProject/audio'
audio_files = [dataset_directory+'/'+f for f in os.listdir(dataset_directory) if f.endswith('.mp3')]
labels = [int(f.split('/audio/')[1].split('_')[0].replace('Q', '')) for f in audio_files]

In [25]:
# Extract features for each audio file
features = [extract_features(file) for file in audio_files]

In [28]:
# Create a DataFrame for the features
columns = (['tempo', 'rms'] +
           [f'chromagram_{i}' for i in range(12)] +
           [f'mel_spectrogram_{i}' for i in range(128)] +
           ['spectral_centroid'] +
           [f'spectral_contrast_{i}' for i in range(7)] +
           ['spectral_rolloff', 'zero_crossing_rate', 'harmonics'] +
           [f'mfcc_{i}' for i in range(20)])

In [31]:
df_features = pd.DataFrame(features, columns=columns)

In [33]:
# Save the features to a CSV file
df_features.to_csv('/home/jovyan/teaching_material/MScProject/audio_features.csv', index=False)

Features extracted and saved to 'audio_features.csv'


In [47]:
y = np.array(labels)

In [49]:
# Perform Min-Max Scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(df_features)

In [53]:
# Perform PCA for dimensionality reduction
pca = PCA(n_components=50)  # Adjust the number of components as needed
X_pca = pca.fit_transform(X_scaled)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [55]:
# Convert labels to categorical
num_classes = len(np.unique(y))
y_train_categorical = to_categorical(y_train - 1, num_classes=num_classes)  # Subtracting 1 to make labels 0-indexed
y_test_categorical = to_categorical(y_test - 1, num_classes=num_classes)

In [58]:
# Reshape data for CNN input (samples, timesteps, features)
X_train_cnn = np.expand_dims(X_train, axis=2)
X_test_cnn = np.expand_dims(X_test, axis=2)

In [62]:
# Build the CNN model
def create_model(optimizer='adam', init_mode='uniform'):
    model = Sequential()
    model.add(Conv1D(128, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    model.add(Conv1D(256, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    model.add(Conv1D(512, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [63]:
# Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model = KerasClassifier(build_fn=create_model, verbose=0)
optimizers = ['adam', 'rmsprop']
init_modes = ['uniform', 'normal']
epochs = [50, 100]
batches = [32, 64]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init_mode=init_modes)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train_cnn, y_train_categorical)

# Best parameters
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

In [64]:
# Train the model
# model.fit(X_train_cnn, y_train_categorical, epochs=100, batch_size=64, validation_data=(X_test_cnn, y_test_categorical))

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_cnn, y_test_categorical)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/50
22/22 [==============================] - 2s 42ms/step - loss: 1.3743 - accuracy: 0.2825 - val_loss: 1.3577 - val_accuracy: 0.3964
Epoch 2/50
22/22 [==============================] - 1s 35ms/step - loss: 1.3399 - accuracy: 0.3964 - val_loss: 1.3301 - val_accuracy: 0.4793
Epoch 3/50
22/22 [==============================] - 1s 35ms/step - loss: 1.2935 - accuracy: 0.4379 - val_loss: 1.2786 - val_accuracy: 0.4970
Epoch 4/50
22/22 [==============================] - 1s 35ms/step - loss: 1.1928 - accuracy: 0.5178 - val_loss: 1.1794 - val_accuracy: 0.4852
Epoch 5/50
22/22 [==============================] - 1s 37ms/step - loss: 1.0492 - accuracy: 0.5562 - val_loss: 1.0608 - val_accuracy: 0.5444
Epoch 6/50
22/22 [==============================] - 1s 37ms/step - loss: 0.9257 - accuracy: 0.6331 - val_loss: 1.0188 - val_accuracy: 0.5799
Epoch 7/50
22/22 [==============================] - 1s 34ms/step - loss: 0.8565 - accuracy: 0.6731 - val_loss: 0.9574 - val_accuracy: 0.6036
Epoch 8/50
22

In [65]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test_cnn, y_test_categorical)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

6/6 [==============================] - 0s 3ms/step - loss: 1.2929 - accuracy: 0.6509
Test Accuracy: 65.09%
